---

# Decision Tree Classifier
 
We consider the problem of predicting health risk of subjects from personal data and habits. We first use for this task a decision tree

![](tree.png)

adapted from the webpage http://www.refactorthis.net/post/2013/04/10/Machine-Learning-tutorial-How-to-create-a-decision-tree-in-RapidMiner-using-the-Titanic-passenger-data-set.aspx. For this exercise sheet, we are required to use only pure Python, and to not import any module, including numpy. In exercise sheet 2, the nearest neighbor part of this exercise sheet will be revisited with numpy.

## Decision Tree Algorithm: Classifying a single instance

Create a function that takes as input a tuple containing values for attributes, and computes the output of the decision tree. Should return a class.
Test the function on the tuple `('yes', 31, 'good')`,

In [1]:
def decision(x):
      
    # Input validation
    assert type(x) == tuple, "Argument is not a tuple"
    assert len(x) == 3, "Arument must have 3 values"
    
    # Divide values into 3 variables for a future operations
    smoker, age, diet = x
    
    if smoker.lower() == "yes":
        if age < 29.5:
            return "less"
        else:
            return "more"
    else:
        if diet.lower() == "good":
            return "less"
        else:
            return "more"

In [2]:
# Test
x = ('yes', 31, 'good')
assert decision(x) == 'more'


## Decision Tree Algorithm: Reading a data set
The file `health-test.txt` contains several fictious records of personal data and habits.

Read the file.Represent the dataset as a list of tuples. Assert the tuples have the same format as above. Extra note of the datatype of each element

In [3]:
def gettest():
    
    # Read the file
    with open('health-test.txt', 'r') as f:
        
        data = list()
        
        for line in f:
            # Split each line into different variables
            smoker, age, diet = line[:-1].split(',')
            
            # Combine all variables into a tuple, with a certain types
            observation = (str(smoker),int(age),str(diet))
            
            # Task: Make extra note of the datatype of each element
            assert type(observation) == tuple
            assert [type(x) for x in observation] == [str,int,str]
            
            # Add the observation to the data list
            data.append(observation)
            
    return data

## Decision Tree Algorithm: Classifying a data set

Application of the decision tree to all points in the dataset, and return the ratio of them that are classified as class 2.

In [4]:
def evaluate_testset():
    
    prediction_list = list()
    
    for observation in gettest():
        
        # Apply Decision Tree to each observation 
        prediction = decision(observation)
        
        # Combine all predictions into one list
        prediction_list.append(prediction)
    
    # Caculate a share of "more"  in the whole prediction list
    ratio = prediction_list.count("more")/len(prediction_list)
    return ratio

## Decision Tree Algorithm: Reading a train set

Suppose that instead of relying on a fixed decision tree, we would like to use a data-driven approach where data points are classified based on a set of training observations manually labeled by experts. Such labeled dataset is available in the file `health-train.txt`. The first three columns have the same meaning than for `health-test.txt`, and the last column corresponds to the labels.
The procedure reads this file and converts it into a list of pairs. The first element of each pair is a triplet of attributes, and the second element is the label.

In [5]:
def gettrain():
    
    # Read the file
    with open('health-train.txt', 'r') as f:
        
        data = list()
        
        for line in f:
            
            # Split each line into different variables
            smoker, age, diet, risk = line[:-1].split(',')
            
            # Combine all variables into a pair of tuple (with 3 values with certain types) and a target variable (as a str)
            pair = [(str(smoker),int(age),str(diet)),str(risk)]
            
            # Add the pair to the data list
            data.append(pair)
            
    return data

## Nearest Neighbor Classifier: Distance function

We consider the nearest neighbor algorithm that classifies test points following the label of the nearest neighbor in the training data. For this, we need to define a distance function between data points. We define it to be

`d(a, b) = (a[0] != b[0]) + ((a[1] - b[1]) / 50.0) ** 2 + (a[2] != b[2])`

where `a` and `b` are two tuples corrsponding to the attributes of two data points. Write a function that retrieves for a test point the nearest neighbor in the training set, and classifies the test point accordingly.

In [6]:
def neighbor(x, trainset):

    # Define distance function
    distance_nnc = lambda a,b: (a[0] != b[0]) + ((a[1] - b[1]) / 50.0) ** 2 + (a[2] != b[2])
    
    prediction = list()
    
    # Extract each pair from the train set
    for observation in trainset:
        
        # Compute the distance
        current_distance = distance_nnc(x,observation[0])
        
        # Check if the current distance lower then the previous one
        if len(prediction) == 0 or current_distance < prediction[0]:
            
            # Update prediction
            prediction = [current_distance,observation[1]]
            
    return prediction[1]

Applicaion of both the decision tree and nearest neighbor classifiers on the test set, and return the list of data point(s) for which the two classifiers disagree, and with which probability it happens.

In [7]:
def compare():
    
    # Get train and test data sets
    train = gettrain()
    test = gettest()
    
    Xdisagree = []
    
    for x in test:
        
        # Apply Decision Tree
        dt_prediction = decision(x)
        
        # Apply Nearest Neighbor Classifiers
        nnc_prediction = neighbor(x, train)
        
        # If disagreement - add x to Xdisagree
        if dt_prediction != nnc_prediction: 
            Xdisagree.append(x)
        
        probability = len(Xdisagree)/len(test)

    return Xdisagree, probability

In [8]:
Xdisagree, probability = compare()
assert type(Xdisagree) == list
assert probability >= 0.0 and probability <= 1.0

One problem of simple nearest neighbors is that one needs to compare the point to predict to all data points in the training set. This can be slow for datasets of thousands of points or more. Alternatively, some classifiers train a model first, and then use it to classify the data.

## Nearest Mean Classifier: Trin and Predict methods

We consider one such trainable model, which operates in two steps:compute the average point for each class and classify new points to be of the class whose average point is nearest to the point to predict. For this classifier, we use the modified distance function:

`d(a,b) = (a[0] - b[0]) ** 2 + ((a[1] - b[1]) / 50.0) ** 2 + (a[2] - b[2]) ** 2`


In [9]:
class NearestMeanClassifier:
        
    def train(self, dataset):
        
        # Define the base vector for "more", and count all "more"
        self.vector_more = [0,0,0]
        n_more = len([x for x in dataset if x[1] == "more"])
        assert n_more != 0, "Not enough target values"

        # Define the base vector for "less", and count all "less"
        self.vector_less = [0,0,0]
        n_less = len([x for x in dataset if x[1] == "less"])
        assert n_less != 0, "Not enough target values"

        for x in dataset:
            
            # Assign each value to a variable, for a future operations
            smoker = x[0][0].lower()
            age = x[0][1]
            diet = x[0][2].lower()
            risk = x[1].lower()

            # Transformation to True (==1) or False (==0)
            observation = (smoker == "yes", age, diet == "poor")

            # Update the vectors
            if risk == "more":
                for i, value in enumerate(self.vector_more):
                    # Compute average by sum of 1/n*Xi
                    self.vector_more[i] += 1/n_more * observation[i]

            if risk == "less":
                for i, value in enumerate(self.vector_less):
                    # Compute average by sum of 1/n*Xi
                    self.vector_less[i] += 1/n_less * observation[i]

        self.mean_vectors = {
            "more": self.vector_more,
            "less": self.vector_less
        }
        
        return self.mean_vectors

    def predict(self, x):
        
        # Define distance function
        distance = lambda a,b: (a[0] - b[0]) ** 2 + ((a[1] - b[1]) / 50.0) ** 2 + (a[2] - b[2]) ** 2
        
        # Transform the new vector
        x = (x[0] == "yes", x[1], x[2] == "poor")
        
        # Find the distance
        distance_to_more = distance(x, self.vector_more)
        distance_to_less = distance(x, self.vector_less)
        
        if abs(distance_to_more) < abs(distance_to_less):
            prediction = "more"
        else:
            prediction = "less"
            
        return prediction

Build an object of class `NearestMeanClassifier`, train it on the training data, and return the mean vector for each class.

In [10]:
def build_and_train():

    nmc = NearestMeanClassifier()
    
    return nmc.train(gettrain())
    
build_and_train()

{'more': [0.5714285714285714, 37.14285714285714, 0.5714285714285714],
 'less': [0.3333333333333333, 32.111111111111114, 0.2222222222222222]}

Predict the test data using the nearest mean classifier and return all test examples for which all three classifiers (decision tree, nearest neighbor and nearest mean) agree. The algorithm returns a list containing tuples.

In [11]:
def predict_test():
    
    test = gettest()
    train = gettrain()
    
    nmc = NearestMeanClassifier()
    nmc.train(train)
    
    agreed_samples = list()
    
    for x in test:
        
        # Decision Tree
        dt_pred = decision(x)
        
        # Nearest neighbor classifier
        nnc_pred = neighbor(x,gettrain())

        # NearestMeanClassifier: 
        nmc_pred = nmc.predict(x)

        if dt_pred == nnc_pred == nmc_pred:
            agreed_samples.append([x,dt_pred])

            
    return agreed_samples
predict_test()

[[('no', 50, 'good'), 'less'],
 [('no', 23, 'good'), 'less'],
 [('yes', 45, 'poor'), 'more'],
 [('no', 60, 'good'), 'less'],
 [('no', 15, 'poor'), 'more'],
 [('no', 18, 'good'), 'less']]